In [ ]:
from bs4 import BeautifulSoup
import requests

import pandas as pd
from datetime import date
from datetime import datetime

In [ ]:
#Empezamos el scraping
#Obtener el HTML
URL_BASE = 'https://cuspide.com/100-mas-vendidos/'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

#"Parsear" ese HTML
soup = BeautifulSoup(html_obtenido, "html.parser")

In [ ]:
#sirve para sacar nombre y url
h3_todos = soup.find_all('h3')
list(h3_todos)
len(h3_todos)

#transformo el contenido de las lista en str, para usar metodos de str
for i,l in enumerate(h3_todos):
    h3_todos[i] = str(l) 
 

In [ ]:
#Se puede sacar el precio, despues del indice 0
bdi_todos = soup.find_all('bdi')
list(bdi_todos)
len(bdi_todos)

#elinimo el primer elemento de la lista
bdi_todos.pop(0)

#transformo el contenido de las lista en str, para usar metodos de str
for i,l in enumerate(bdi_todos):
    bdi_todos[i] = str(l) 


In [ ]:
#separa los str para sacar la informacion
nom_url=[]
for i,l in enumerate(h3_todos):
    nom_url.append(h3_todos[i][l.find('href=')+5 :])
    
     


In [ ]:
#URL
#saca la url de la lista de string
url_=[]
for i,l in enumerate(nom_url):
    url_.append(nom_url[i][ :l.find('>')])

#quita la comilla doble del string del url
for i,l in enumerate(url_):    
    url_[i] = l.replace('"','')
   

In [ ]:
#Consigo los nombres de los libros
nombre=[]
for i,l in enumerate(nom_url):
    nombre.append(nom_url[i][l.find('>')+1 : l.find('<')])

In [ ]:
#se tiene el valor de los precios, pero en notacion latina 0.000,00
precio=[]
for i,l in enumerate(bdi_todos):
    precio.append(bdi_todos[i][l.find('\xa0')+1 : l.rfind('<')])

#paso el precion a notacion internacional y transformo en float
for i,l in enumerate(precio):
     precio[i] = float(l.replace('.','').replace(',','.'))    


In [ ]:
#Para la fecha 16/02/24, existen 8 libros sin precio en la pagina
#por lo que decidio colocarlos 0 

# precio.insert(7,0)
# precio.insert(18,0)
# precio.insert(22,0)
# precio.insert(26,0)
# precio.insert(58,0)
# precio.insert(67,0)
# precio.insert(80,0)
# precio.insert(88,0)
# precio.insert(95,0)



In [ ]:
#crea el datafreme a partir de las listas
df = pd.DataFrame(dict(titulo=nombre,url=url_,precio_pesos=precio))

In [ ]:
#scraping para la transformacion de pesos argentinos a dolares
#Obtener el HTML
URL_BASE1 = 'https://www.google.com/search?q=pesos+argentinos+a+dolares&rlz=1C1CHBD_esCO1034CO1034&oq=pesos+argentinos+&gs_lcrp=EgZjaHJvbWUqCQgAECMYJxiKBTIJCAAQIxgnGIoFMgYIARBFGDkyBwgCEAAYgAQyBwgDEAAYgAQyBwgEEAAYgAQyBwgFEAAYgAQyBwgGEAAYgAQyBwgHEAAYgAQyBwgIEAAYgAQyBwgJEAAYgATSAQg0OTAwajFqN6gCALACAA&sourceid=chrome&ie=UTF-8'
pedido_obtenido1 = requests.get(URL_BASE1)
html_obtenido1 = pedido_obtenido1.text

#"Parsear" ese HTML
soup1 = BeautifulSoup(html_obtenido1, "html.parser")

#sirve para sacar nombre y url
values_todos = soup1.find_all('h2')
list(values_todos)
len(values_todos)



In [ ]:
#scraping para la transformacion de pesos argentinos a dolares
#Obtener el HTML
URL_BASE1 = 'https://www.xe.com/es/currencyconverter/convert/?Amount=1&From=ARS&To=USD'
pedido_obtenido1 = requests.get(URL_BASE1)
html_obtenido1 = pedido_obtenido1.text

#"Parsear" ese HTML
soup1 = BeautifulSoup(html_obtenido1, "html.parser")

#sirve para sacar nombre y url
body = soup1.find_all('body')
list(body)
body[0] = str(body[0])


#toma la parte decimal 0.00
indice = int(body[0].find('"result__BigRate-sc-1bsijpp-1 iGrAod"'))

a = body[0][indice + 38 : indice + 42]
#toma los decimaneles luego del 0.00
b = body[0][indice + 69: indice + 75]
#viene en formato latino, toca pasarlo a formato de usa
a = a.replace(',','.')

#concadena los calores y los vuelve un string
ars_usd = float(a+b)

In [ ]:
#scraping para la transformacion de dolares a euros
#Obtener el HTML
URL_BASE1 = 'https://www.xe.com/es/currencyconverter/convert/?Amount=1&From=USD&To=EUR'
pedido_obtenido1 = requests.get(URL_BASE1)
html_obtenido1 = pedido_obtenido1.text

#"Parsear" ese HTML
soup1 = BeautifulSoup(html_obtenido1, "html.parser")

#sirve para sacar nombre y url
body = soup1.find_all('body')
list(body)
body[0] = str(body[0])

#toma la parte decimal 0.00
indice = int(body[0].find('"result__BigRate-sc-1bsijpp-1 iGrAod"'))

a = body[0][indice + 38 : indice + 42]
#toma los decimaneles luego del 0.00
b = body[0][indice + 69: indice + 74]
#viene en formato latino, toca pasarlo a formato de usa
a = a.replace(',','.')

#concadena los calores y los vuelve un string
usd_eur = float(a+b)

In [ ]:
#agregar columna de precio en dolares y euros
df['precio_dolar'] = df['precio_pesos']*ars_usd
df['precio_euro'] = df['precio_dolar']*usd_eur

In [ ]:

df['precio_dolar'] = df['precio_dolar'].round(2)
df['precio_euro'] = df['precio_euro'].round(2)

In [ ]:

#Agregar la fecha
now = datetime.now()
#format = now.strftime('%d/%m/%Y')

df['fecha'] = now.strftime('%d/%m/%Y')


In [ ]:
df.to_csv('toplibrosv1.csv', index=False)